<a href="https://colab.research.google.com/github/kdChen07/progress-and-problems/blob/main/%E5%8F%AF%E9%80%89POI%E4%B8%8E%E6%97%A0%E7%96%AB%E8%8B%97%E6%9C%8D%E5%8A%A1MSOA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#计算可以选择的潜在POI与无服务的MSAO

In [ ]:
!pip install access
!pip install logging
!pip install geopandas
!pip install mapclassify
!pip install legendgram

In [ ]:
from access import access, weights, datasets
import logging
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify as mc
from legendgram import legendgram
import palettable.matplotlib as palmpl
import seaborn as sns
import numpy as np
from scipy import stats
from google.colab import files
from shapely import wkt

#导入数据

In [7]:
MSOA=pd.read_csv('https://github.com/kdChen07/progress-and-problems/blob/main/data/Boundary%20MSOAs%20Lack%20of%20Vaccination%20Service.csv?raw=ture')
MSOA.head(1)

,Unnamed: 0,MSOA11CD,MSOA11NM,pop0-49,pop50-59,pop60-69,pop70-79,pop80-89,pop90+,total_pop,geometry,MSOA,MSOA_with_supply,dissolve
0,1,E02002527,Redcar and Cleveland 013,3094,709,573,484,159,59,5078,"POLYGON ((471385.4547182356 520187.9116093202,...",False,False,1


In [8]:
MSOA['geometry'] = MSOA['geometry'].apply(wkt.loads)
MSOA_bou = gpd.GeoDataFrame(MSOA, geometry='geometry')
MSOA_bou.crs = 'epsg:27700'


In [9]:
MSOA_bou

,Unnamed: 0,MSOA11CD,MSOA11NM,pop0-49,pop50-59,pop60-69,pop70-79,pop80-89,pop90+,total_pop,geometry,MSOA,MSOA_with_supply,dissolve
0,1,E02002527,Redcar and Cleveland 013,3094,709,573,484,159,59,5078,"POLYGON ((471385.455 520187.912, 471252.707 51...",False,False,1
1,2,E02002692,East Riding of Yorkshire 009,4692,1490,1303,1090,396,108,9079,"POLYGON ((474512.215 447982.734, 474236.819 44...",False,False,1
2,3,E02002693,East Riding of Yorkshire 010,3247,1271,1167,1040,372,59,7156,"POLYGON ((517327.024 458121.170, 517434.213 45...",False,False,1
3,4,E02002694,East Riding of Yorkshire 011,6305,1925,1585,1479,694,136,12124,"POLYGON ((484978.621 457448.011, 483598.543 45...",False,False,1
4,5,E02002695,East Riding of Yorkshire 012,3631,1197,1366,1367,628,116,8305,"POLYGON ((525046.515 440638.080, 523460.549 44...",False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,379,E02006845,North Somerset 026,6772,1147,884,672,320,102,9897,"POLYGON ((341421.830 162151.855, 340439.126 16...",False,False,1
379,380,E02006846,North Somerset 027,7366,955,506,288,117,17,9249,"POLYGON ((336349.281 161524.325, 335072.481 16...",False,False,1
380,381,E02006857,Eastbourne 013,4821,1221,1108,1153,521,134,8958,"POLYGON ((562630.620 102435.229, 562809.459 10...",False,False,1
381,382,E02006870,Ryedale 008,8386,2059,1673,1378,799,190,14485,"POLYGON ((482366.113 475716.224, 482873.309 47...",False,False,1


In [10]:
MSOA_point=pd.read_csv('https://github.com/kdChen07/progress-and-problems/blob/main/data/Point%20MSOAs%20Lack%20of%20Vaccination%20Service.csv?raw=ture')
MSOA_point.head(1)

,index,objectid,msoa11cd,geometry,MSOA,MSOA_with_supply
0,1,311,E02002527,POINT (471554.0053845677 518513.6276244508),False,False


In [11]:
MSOA_point['geometry'] = MSOA_point['geometry'].apply(wkt.loads)
MSOA_lack = gpd.GeoDataFrame(MSOA_point, geometry='geometry')
MSOA_lack.crs = 'epsg:27700'
#MSOA_lack.crs = 'epsg:4326'

In [12]:
MSOA_lack = MSOA_lack.to_crs('epsg:4326')
MSOA_lack['point_lon'] = MSOA_lack['geometry'].x
MSOA_lack['point_lat'] = MSOA_lack['geometry'].y

In [13]:
MSOA_lack

,index,objectid,msoa11cd,geometry,MSOA,MSOA_with_supply,point_lon,point_lat
0,1,311,E02002527,POINT (-0.89510 54.55681),False,False,-0.895104,54.556813
1,2,1636,E02002692,POINT (-0.90041 53.97980),False,False,-0.900405,53.979796
2,3,1634,E02002693,POINT (-0.27102 53.94519),False,False,-0.271017,53.945190
3,4,1632,E02002694,POINT (-0.77820 53.93154),False,False,-0.778200,53.931540
4,5,1631,E02002695,POINT (-0.16898 53.90988),False,False,-0.168981,53.909875
...,...,...,...,...,...,...,...,...
378,379,6413,E02006845,POINT (-2.90642 51.35837),False,False,-2.906422,51.358374
379,380,6410,E02006846,POINT (-2.92958 51.34798),False,False,-2.929576,51.347977
380,381,5000,E02006857,POINT (0.31312 50.78637),False,False,0.313122,50.786367
381,382,1484,E02006870,POINT (-0.78674 54.13234),False,False,-0.786744,54.132341


In [ ]:
#MSOA_lack_df

,index,objectid,msoa11cd,point_lon,point_lat
0,1,311,E02002527,-0.895104,54.556813
1,2,1636,E02002692,-0.900405,53.979796
2,3,1634,E02002693,-0.271017,53.945190
3,4,1632,E02002694,-0.778200,53.931540
4,5,1631,E02002695,-0.168981,53.909875
...,...,...,...,...,...
378,379,6413,E02006845,-2.906422,51.358374
379,380,6410,E02006846,-2.929576,51.347977
380,381,5000,E02006857,0.313122,50.786367
381,382,1484,E02006870,-0.786744,54.132341


In [15]:
df=pd.read_csv('https://github.com/kdChen07/progress-and-problems/blob/main/data/poi_three_types1.csv?raw=ture')
df.head(1)

,index,name,feature_easting,feature_northing,pointx_class
0,1,Reiver House,338714.0,556014.0,5280365


In [16]:
point = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.feature_easting, df.feature_northing))
point.crs = 'epsg:27700'

In [17]:
point = point.to_crs('epsg:4326')
point['point_lon'] = point['geometry'].x
point['point_lat'] = point['geometry'].y

In [18]:
point

,index,name,feature_easting,feature_northing,pointx_class,geometry,point_lon,point_lat
0,1,Reiver House,338714.0,556014.0,5280365,POINT (-2.95709 54.89508),-2.957088,54.895084
1,2,Lloyds Pharmacy,430296.0,279574.0,5280364,POINT (-1.55602 52.41333),-1.556017,52.413332
2,3,Wilson Hospital,527876.0,168073.0,5280371,POINT (-0.16325 51.39729),-0.163246,51.397288
3,4,Elland Pharmacy,410903.0,421112.0,5280364,POINT (-1.83639 53.68635),-1.836388,53.686351
4,5,Pimhole Pharmacy,381373.0,410753.0,5280364,POINT (-2.28289 53.59302),-2.282894,53.593018
...,...,...,...,...,...,...,...,...
21807,21808,Ultralase,318232.0,176550.0,5280365,POINT (-3.17891 51.48197),-3.178908,51.481971
21808,21809,Fadeaway Laser Clinic,444081.0,111092.0,5280365,POINT (-1.37456 50.89763),-1.374558,50.897631
21809,21810,Midland Eye,415348.0,280208.0,5280365,POINT (-1.77575 52.41965),-1.775749,52.419652
21810,21811,Optimax Laser Eye Clinic,533193.0,181452.0,5280365,POINT (-0.08183 51.51630),-0.081829,51.516296


In [19]:
point_df = point[['index','name','point_lon','point_lat']]
point_df

,index,name,point_lon,point_lat
0,1,Reiver House,-2.957088,54.895084
1,2,Lloyds Pharmacy,-1.556017,52.413332
2,3,Wilson Hospital,-0.163246,51.397288
3,4,Elland Pharmacy,-1.836388,53.686351
4,5,Pimhole Pharmacy,-2.282894,53.593018
...,...,...,...,...
21807,21808,Ultralase,-3.178908,51.481971
21808,21809,Fadeaway Laser Clinic,-1.374558,50.897631
21809,21810,Midland Eye,-1.775749,52.419652
21810,21811,Optimax Laser Eye Clinic,-0.081829,51.516296


In [22]:
distance_cost_df = pd.read_csv('https://github.com/kdChen07/progress-and-problems/blob/main/data/distance18994.csv?raw=ture') 
distance_cost_df = distance_cost_df.rename(columns={"Distance in meter": "cost", "origin_id": "origin", "destination_id": "dest","Unnamed: 0":"index"})
distance_cost_df

,index,cost,origin,dest
0,0,17438.2,E02002527,124
1,1,15669.6,E02002527,4155
2,2,608.3,E02002527,4159
3,3,16577.0,E02002527,4162
4,4,11904.9,E02002527,4609
...,...,...,...,...
18989,18989,4303.6,E02006880,18563
18990,18990,1405.0,E02006880,18686
18991,18991,19443.3,E02006880,18900
18992,18992,6235.8,E02006880,19549


#筛选距离小于10英里的距离

In [23]:
distance_cost_df_test = distance_cost_df.loc[distance_cost_df['cost']<16093.44]
distance_cost_df_test

,index,cost,origin,dest
1,1,15669.6,E02002527,4155
2,2,608.3,E02002527,4159
4,4,11904.9,E02002527,4609
5,5,14355.9,E02002527,5050
7,7,989.8,E02002527,5056
...,...,...,...,...
18986,18986,6549.4,E02006880,17608
18989,18989,4303.6,E02006880,18563
18990,18990,1405.0,E02006880,18686
18992,18992,6235.8,E02006880,19549


In [24]:
a = pd.unique(distance_cost_df_test['origin'])

# Set 'MSOA_with_supply' field True if a place has at least one vaccination site within 10 miles of distance, otherwise False.
b = pd.DataFrame()
b['MSOA'] = a
b['MSOA_with_supply'] = True

MSOA_with_supply = pd.merge(left = MSOA_point,right = b, how='left', left_on='msoa11cd', right_on='MSOA')
MSOA_with_supply=MSOA_with_supply.fillna(False)

In [25]:
MSOA_with_supply

,index,objectid,msoa11cd,geometry,MSOA_x,MSOA_with_supply_x,MSOA_y,MSOA_with_supply_y
0,1,311,E02002527,POINT (471554.005 518513.628),False,False,E02002527,True
1,2,1636,E02002692,POINT (472211.262 454307.961),False,False,E02002692,True
2,3,1634,E02002693,POINT (513578.580 451282.210),False,False,E02002693,True
3,4,1632,E02002694,POINT (480317.886 449070.500),False,False,E02002694,True
4,5,1631,E02002695,POINT (520376.509 447521.787),False,False,E02002695,True
...,...,...,...,...,...,...,...,...
378,379,6413,E02006845,POINT (336983.216 162534.986),False,False,E02006845,True
379,380,6410,E02006846,POINT (335356.443 161398.909),False,False,E02006846,True
380,381,5000,E02006857,POINT (563147.308 101082.989),False,False,E02006857,True
381,382,1484,E02006870,POINT (479374.087 471401.343),False,False,E02006870,True


In [26]:
distance_cost_df_test

,index,cost,origin,dest
1,1,15669.6,E02002527,4155
2,2,608.3,E02002527,4159
4,4,11904.9,E02002527,4609
5,5,14355.9,E02002527,5050
7,7,989.8,E02002527,5056
...,...,...,...,...
18986,18986,6549.4,E02006880,17608
18989,18989,4303.6,E02006880,18563
18990,18990,1405.0,E02006880,18686
18992,18992,6235.8,E02006880,19549


In [27]:
point_df

,index,name,point_lon,point_lat
0,1,Reiver House,-2.957088,54.895084
1,2,Lloyds Pharmacy,-1.556017,52.413332
2,3,Wilson Hospital,-0.163246,51.397288
3,4,Elland Pharmacy,-1.836388,53.686351
4,5,Pimhole Pharmacy,-2.282894,53.593018
...,...,...,...,...
21807,21808,Ultralase,-3.178908,51.481971
21808,21809,Fadeaway Laser Clinic,-1.374558,50.897631
21809,21810,Midland Eye,-1.775749,52.419652
21810,21811,Optimax Laser Eye Clinic,-0.081829,51.516296


In [28]:
c = pd.unique(distance_cost_df_test['dest'])
d = pd.DataFrame()
d['point'] = c
d['opt_points'] = True

opt_points = pd.merge(left = point_df,right = d,how='left',left_on='index',right_on='point')
opt_points = opt_points.fillna(False)

In [29]:
opt_points

,index,name,point_lon,point_lat,point,opt_points
0,1,Reiver House,-2.957088,54.895084,False,False
1,2,Lloyds Pharmacy,-1.556017,52.413332,False,False
2,3,Wilson Hospital,-0.163246,51.397288,False,False
3,4,Elland Pharmacy,-1.836388,53.686351,False,False
4,5,Pimhole Pharmacy,-2.282894,53.593018,False,False
...,...,...,...,...,...,...
21807,21808,Ultralase,-3.178908,51.481971,False,False
21808,21809,Fadeaway Laser Clinic,-1.374558,50.897631,False,False
21809,21810,Midland Eye,-1.775749,52.419652,False,False
21810,21811,Optimax Laser Eye Clinic,-0.081829,51.516296,False,False


In [30]:
opt_points.to_csv('opt_points.csv')

In [31]:
files.download('opt_points.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>